### Imports 

In [2]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime
import pandas as pd

### Transforms 

In [3]:
df = pd.read_csv('./data/profile-info.csv')
df

,active_since,total_purchases,total_reviews,purchases_last_60_days,reviews_last_60_days,country,age_group
0,11-02-2015,35,3,3,1,india,18-24
1,04-23-2011,47,23,2,0,portugal,65+
2,01-13-2014,34,2,1,0,usa,35-44
3,12-16-2022,2,72,1,67,china,55-64
4,04-04-2022,10,34,10,32,usa,25-34
5,10-19-2021,345,24,82,14,usa,35-44
6,11-04-2019,32,2,2,1,canada,45-54
7,03-08-2020,34,23,1,1,canada,18-24
8,04-03-2019,234,24,2,5,spain,25-34
9,09-04-2022,24,45,2,10,belgium,18-24


In [53]:
def days_since(date_str):
    date = datetime.strptime(date_str, '%m-%d-%Y')
    now = datetime.now()
    return (now - date).days

In [54]:
transformer = FunctionTransformer(days_since, validate=False)
transformer.transform('03-03-2022')

286

In [55]:
# Define the categories and their ordering
age_groups = [['18-24'], ['25-34'], ['35-44'], ['45-54'], ['55-64'], ['65+']]
# Create the encoder
age_group_encoder = OrdinalEncoder().fit(df[['age_group']])
age_group_encoder.categories

'auto'

In [57]:
# Use the fitted encoder to transform new data
test_age_group = ['35-44']
encoded_data = age_group_encoder.transform([test_age_group])
encoded_data

array([[2.]])